In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt

with open("all_roi_timeseries_denoised.json", "r") as f:
    ts_dict = json.load(f)

mouse_names = sorted(ts_dict.keys())
all_mice_ts = []
all_mice_roi_ids = []

for m in mouse_names:
    roi_ids = ts_dict[m]['roi_ids']
    ts = np.array(ts_dict[m]['timeseries']) 
    all_mice_ts.append(ts)
    all_mice_roi_ids.append(roi_ids)

assert all(roi_ids == all_mice_roi_ids[0] for roi_ids in all_mice_roi_ids)
roi_id_to_idx = {roi_id: idx for idx, roi_id in enumerate(all_mice_roi_ids[0])}

merged_rois = {
    'RSPd':   [326, 327, 329, 330, 331],
    'ACAd':   [227, 228, 229, 230, 231],
    'PL':     [239, 241, 242, 243, 244],
    'ILA':    [246, 248, 249, 250, 251],
    'ORBl':   [259, 260, 261, 262, 263],
    'ORBmc':  [265, 267, 268, 269, 270],
    'ORBvl':  [273, 274, 275, 276, 277],
    'RSPv':   [333, 335, 336, 337, 338],
    'RSPagl': [299, 300, 301, 302, 303],
    'TEa':    [361, 362, 363, 364, 365, 366],
    # Basal Forebrain
    'MedialSeptal':       [617],
    'DiagonalBand':       [618],
    'SubstantiaInnominata':[613],
    'VentralPallidum':    [608]
}
merged_roi_names = list(merged_rois.keys())

merged_indices = {}
for region, roi_id_list in merged_rois.items():
    merged_indices[region] = [roi_id_to_idx[roi_id] for roi_id in roi_id_list]

merged_ts_allmice = []
for mouse_ts in all_mice_ts:
    merged = []
    for roi_idx_list in merged_indices.values():
        roi_ts = mouse_ts[roi_idx_list, :]
        merged.append(np.mean(roi_ts, axis=0))
    merged_ts_allmice.append(np.stack(merged, axis=0))
merged_ts_allmice = np.stack(merged_ts_allmice, axis=0)

print("Shape (n_mice, n_merged_rois, T):", merged_ts_allmice.shape)
print("ROI order:", merged_roi_names)

np.save("merged_dmn_bf_timeseries.npy", merged_ts_allmice)
with open("merged_roi_names.txt", "w") as f:
    f.write('\n'.join(merged_roi_names))

merged_ts_allmice = np.load("merged_dmn_bf_timeseries.npy")

n_mice, n_rois, T = merged_ts_allmice.shape
fc_matrices = np.zeros((n_mice, n_rois, n_rois))

for i in range(n_mice):
    data = merged_ts_allmice[i]
    fc = np.corrcoef(data)
    fc_matrices[i] = fc

np.save("fc_matrices_merged_dmn_bf.npy", fc_matrices)
print("FC matrices saved. Shape:", fc_matrices.shape)

fc_matrices = np.load("fc_matrices_merged_dmn_bf.npy")
with open("merged_roi_names.txt") as f:
    merged_roi_names = [line.strip() for line in f.readlines()]

mean_fc = np.mean(fc_matrices, axis=0)

plt.figure(figsize=(8, 8))
plt.imshow(mean_fc, vmin=-1, vmax=1, cmap='bwr')
plt.colorbar(label="Pearson r")
plt.xticks(range(len(merged_roi_names)), merged_roi_names, rotation=90)
plt.yticks(range(len(merged_roi_names)), merged_roi_names)
plt.title("Average FC Matrix (DMN + BF)")
plt.tight_layout()
plt.show()